In [138]:
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm

In [164]:
GATEWAY_HOST = 'https://www.binance.com'
API_PATH = '/'.join([GATEWAY_HOST, 'bapi/futures/v1'])

endpoints = {
    "positions" : {
        "path": 'public/future/copy-trade/lead-data/positions?portfolioId=%s', 
        "type": "simple",
        "params": {}
        },
    "performance" : {
        "path": 'public/future/copy-trade/lead-portfolio/performance?portfolioId=%s&timeRange=%s',
        "type": "simple",
        "params": {"timeRange": "90D"}
        },
    "detail" : {
        "path": 'friendly/future/copy-trade/lead-portfolio/detail?portfolioId=%s', 
        "type": "simple",
        "params": {}
    },
    "chart" : {
        "path": 'public/future/copy-trade/lead-portfolio/chart-data?portfolioId=%s&timeRange=%s&dataType=%s', 
        "type": "simple",
        "params": {"timeRange": "90D", "dataType": "ROI"}
    },
    "position_history" : {
        "path": 'public/future/copy-trade/lead-portfolio/position-history', 
        "type": "paginated",
        "params": {"pageNumber" : 1, "pageSize": 10}
    },
    "transfer_history" : {
        "path": 'public/future/copy-trade/lead-portfolio/transfer-history', 
        "type": "paginated",
        "params": {"pageNumber" : 1, "pageSize": 10}
    }
}


async def fetch_data(leaderId, endpointType, params={}):
    endpoint = endpoints[endpointType]
    # Filter out the empty params
    filtered_params = {}
    for default_key, default_value in endpoint['params'].items():
        if default_key in params.keys() and params[default_key] is not None:
            filtered_params[default_key] = params[default_key]
        else:
            filtered_params[default_key] = default_value

    if endpoint["type"] == 'simple':
        # Interpolate strings
        path = endpoint["path"] % (leaderId, *filtered_params.values())
        url = '/'.join([API_PATH, path])

        response = requests.get(url)

    if endpoint["type"] == 'paginated':
        url = '/'.join([API_PATH, endpoint["path"]])

        response = requests.post(
            url,
            json={"portfolioId": leaderId} | filtered_params,
            )

    return response.json()


async def fetch_pages(leaderId, endpointType, params={}, page_number=None, result=None, latest_item=None, reference=None, progress_bar=None):
    if page_number is None:
        page_number = 1
    if result is None:
        result = []

    response = await fetch_data(leaderId, endpointType, {"pageNumber": page_number} | params)

    if response["success"]:
        response_data = response["data"]
        response_list = response_data["list"]

        if latest_item and reference:
            response_list = sorted(response_list, key=lambda x: x[reference], reverse=True)
            item_index = 0

            for item in response_list:
                if item[reference] > latest_item[reference]:
                    result.append(item)
                    item_index += 1

                    if item_index == 10:
                        next_page = page_number + 1
                        return await fetch_pages(leaderId, endpointType, params, next_page, result, latest_item, reference)
                else:
                    # print({ "success": True, "reason": "partial", "message": f"Fetched pages {endpointType} - finished by update", "data": result })
                    return result
        else:
            result += response_list
            total_n_results = response_data["total"]
            pages_length = total_n_results // 10

            if progress_bar is None:
                progress_bar = tqdm(total=total_n_results)

            progress_bar.update(len(response_list))
            # If remainder, add another page
            if total_n_results % 10:
                pages_length += 1

            if page_number <= pages_length:
                next_page = page_number + 1

                return await fetch_pages(leaderId, endpointType, params, next_page, result, progress_bar=progress_bar)
            else:
                return result
    
    else:
        return { "success": False, "message": f"Could not fetch page {page_number}/{pages_length} of {endpointType}" }
    

In [220]:
# leader_ids = ["3846188874749232129", "3842534998056366337"]
leader_ids = ["3846188874749232129"]
leader = {}

for leader_id in leader_ids:
    positions = await fetch_data(leader_id, 'positions')
    leader_positions = pd.DataFrame(positions["data"])
    leader_positions["LEADER_ID"] = leader_id
    
    position_history = await fetch_pages(leader_id, 'position_history')
    leader_position_history = pd.DataFrame(position_history)
    leader_position_history["LEADER_ID"] = leader_id

    transfer_history = await fetch_pages(leader_id, 'transfer_history')
    leader_transfer_history = pd.DataFrame(transfer_history)
    leader_transfer_history["LEADER_ID"] = leader_id

leader_positions.head()

100%|██████████| 11/11 [00:00<00:00, 17.03it/s]


,id,symbol,collateral,positionAmount,entryPrice,unrealizedProfit,cumRealized,askNotional,bidNotional,notionalValue,markPrice,leverage,isolated,isolatedWallet,adl,positionSide,breakEvenPrice,LEADER_ID
0,0_SUSHIUSDT_BOTH,SUSHIUSDT,USDT,0,0.0,0.00000000,0,0,0,0,1.53651051,1,True,0,0,BOTH,0.0,3846188874749232129
1,0_SUSHIUSDT_LONG,SUSHIUSDT,USDT,0,0.0,0.00000000,0,0,0,0,1.53651051,1,True,0,0,LONG,0.0,3846188874749232129
2,0_SUSHIUSDT_SHORT,SUSHIUSDT,USDT,0,0.0,0.00000000,-422.08230134,0,0,0,1.53651051,1,True,0,0,SHORT,0.0,3846188874749232129
3,0_BNBUSDT_BOTH,BNBUSDT,USDT,0.00,0.0,0.00000000,0,0,0,0,609.57467690,1,True,0,0,BOTH,0.0,3846188874749232129
4,0_BNBUSDT_LONG,BNBUSDT,USDT,0.00,0.0,0.00000000,0,0,0,0,609.57467690,1,True,0,0,LONG,0.0,3846188874749232129


In [221]:
leader_transfer_history["INCOMING"] = leader_transfer_history["to"] == 'Lead Trading Account'
leader_transfer_history["NOTIONAL_VALUE"] = leader_transfer_history.apply(lambda row: row["amount"] if row["INCOMING"] else -row["amount"], axis=1)

leader_transfer_history.head()

,time,coin,amount,from,to,transType,LEADER_ID,INCOMING,NOTIONAL_VALUE
0,1711015734693,USDT,62523.139709,Fiat and Spot,Lead Trading Account,LEAD_DEPOSIT,3846188874749232129,True,62523.139709
1,1710910426929,USDT,2109.074057,Fiat and Spot,Lead Trading Account,LEAD_DEPOSIT,3846188874749232129,True,2109.074057
2,1710631903556,USDT,48177.912951,Lead Trading Account,Fiat and Spot,LEAD_WITHDRAW,3846188874749232129,False,-48177.912951
3,1710631606472,USDT,48177.912951,Fiat and Spot,Lead Trading Account,LEAD_DEPOSIT,3846188874749232129,True,48177.912951
4,1710631512228,USDT,48175.731055,Lead Trading Account,Fiat and Spot,LEAD_WITHDRAW,3846188874749232129,False,-48175.731055


In [222]:
leader_position_history

,id,symbol,type,opened,closed,avgCost,avgClosePrice,closingPnl,maxOpenInterest,closedVolume,isolated,side,status,updateTime,LEADER_ID
0,28831399,BCHUSDT,UM,1712752315689,1.712785e+12,603.280501,618.462375,-945.041280,62.248,62.248,Isolated,Short,All Closed,1712784529622,3846188874749232129
1,28675086,BCHUSDT,UM,1712711525798,1.712729e+12,666.038565,626.704920,2201.700750,55.975,55.975,Isolated,Short,All Closed,1712728550720,3846188874749232129
2,28368782,ETHUSDT,UM,1712627977755,NaN,3603.292522,3490.719604,-6972.078457,45.070,44.950,Isolated,Long,Partially Closed,1712784543076,3846188874749232129
3,27731385,BCHUSDT,UM,1712373128871,1.712461e+12,702.900561,684.506554,905.592130,49.233,49.233,Isolated,Short,All Closed,1712461052390,3846188874749232129
4,27604253,ETHUSDT,UM,1712319921263,1.712628e+12,3296.424227,3565.913762,16574.414869,41.217,61.503,Isolated,Long,All Closed,1712665128410,3846188874749232129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,10433832,ETHUSDT,UM,1707831600369,1.707834e+12,2641.025157,2610.031900,-2356.820250,76.043,76.043,Isolated,Long,All Closed,1707834104910,3846188874749232129
96,10225346,ETHUSDT,UM,1707732084449,1.707831e+12,2483.687340,2576.053026,20238.614850,146.525,219.114,Isolated,Long,All Closed,1707831450716,3846188874749232129
97,9987905,ETHUSDT,UM,1707565217287,1.707732e+12,2479.567046,2473.923819,-832.652429,147.549,147.549,Isolated,Long,All Closed,1707731638248,3846188874749232129
98,9450605,ETHUSDT,UM,1707209329727,1.707565e+12,2334.502083,2476.586570,19930.475049,140.272,140.272,Cross,Long,All Closed,1707565060277,3846188874749232129


In [223]:
sum_columns = [
    "positionAmount",
    "unrealizedProfit",
    "cumRealized",
    "notionalValue",
	"markPrice",
    "ABSOLUTE_VALUE"
]

average_columns = [
    "entryPrice",
	"markPrice",
	"leverage",
	"breakEvenPrice",
]

drop_columns = [
    "id",
    "collateral",
    "isolated",
    "isolatedWallet",
    "adl",
    "askNotional",
    "bidNotional"
]

filtered_leader_positions = leader_positions.apply(lambda column: column.astype(float) if column.name in sum_columns + average_columns else column)
filtered_leader_positions = filtered_leader_positions.loc[(filtered_leader_positions["positionAmount"] != 0) | (filtered_leader_positions["collateral"] != "USDT")]
filtered_leader_positions = filtered_leader_positions.drop(columns=drop_columns)
filtered_leader_positions["ABSOLUTE_VALUE"] = abs(filtered_leader_positions["notionalValue"] / filtered_leader_positions["leverage"])
filtered_leader_positions

,symbol,positionAmount,entryPrice,unrealizedProfit,cumRealized,notionalValue,markPrice,leverage,positionSide,breakEvenPrice,LEADER_ID,ABSOLUTE_VALUE
13,ETHUSDT,43.252,3559.088189,71.011628,95554.626641,154008.69396,3560.73,2.0,LONG,3725.772762,3846188874749232129,77004.34698


In [234]:

def aggregate_positions(group: pd.DataFrame) -> pd.Series:
    result = {}
    
    for column in sum_columns: result[column] = np.sum(group[column])
    for column in average_columns: result[column] = np.average(group[column], weights=group["positionAmount"])

    if len(group) > 1: result["positionSide"] = "BOTH"
    else: result["positionSide"] = group["positionSide"].values[0]

    # result["LEADER_IDS"] = list(set(group["LEADER_ID"]))
    return pd.Series(result)

grouped_leader_positions = filtered_leader_positions.groupby("symbol").apply(aggregate_positions, include_groups=False)
grouped_leader_positions = grouped_leader_positions.rename(columns={key: key + "_SUM" for key in sum_columns} | {key: key + "_AVERAGE" for key in average_columns})
grouped_leader_positions["POSITION_SHARE"] = grouped_leader_positions["ABSOLUTE_VALUE_SUM"] / np.sum(grouped_leader_positions["ABSOLUTE_VALUE_SUM"])

grouped_leader_positions

,positionAmount_SUM,unrealizedProfit_SUM,cumRealized_SUM,notionalValue_SUM,markPrice_AVERAGE,ABSOLUTE_VALUE_SUM,entryPrice_AVERAGE,leverage_AVERAGE,breakEvenPrice_AVERAGE,positionSide,POSITION_SHARE
symbol,,,,,,,,,,,
ETHUSDT,43.252,71.011628,95554.626641,154008.69396,3560.73,77004.34698,3559.088189,2.0,3725.772762,LONG,1.0


In [228]:
transfer_balance = np.sum(leader_transfer_history["NOTIONAL_VALUE"])
historic_PNL = np.sum(leader_position_history["closingPnl"])
total_balance = transfer_balance + historic_PNL
leader["positions"] = filtered_leader_positions.to_dict()
leader["mix"] = grouped_leader_positions.to_dict()
leader["account"] = {
    "transfer_balance": transfer_balance,
    "historic_PNL": historic_PNL,
    "total_balance": total_balance

}

leader

{'positions': {'symbol': {13: 'ETHUSDT'},
  'positionAmount': {13: 43.252},
  'entryPrice': {13: 3559.08818858},
  'unrealizedProfit': {13: 71.01162753},
  'cumRealized': {13: 95554.62664121},
  'notionalValue': {13: 154008.69396},
  'markPrice': {13: 3560.73},
  'leverage': {13: 2.0},
  'positionSide': {13: 'LONG'},
  'breakEvenPrice': {13: 3725.772762335},
  'LEADER_ID': {13: '3846188874749232129'},
  'ABSOLUTE_VALUE': {13: 77004.34698}},
 'mix': {'positionAmount': {'ETHUSDT': 43.252},
  'unrealizedProfit': {'ETHUSDT': 71.01162753},
  'cumRealized': {'ETHUSDT': 95554.62664121},
  'notionalValue': {'ETHUSDT': 154008.69396},
  'markPrice': {'ETHUSDT': 3560.73},
  'ABSOLUTE_VALUE': {'ETHUSDT': 77004.34698},
  'entryPrice': {'ETHUSDT': 3559.08818858},
  'leverage': {'ETHUSDT': 2.0},
  'breakEvenPrice': {'ETHUSDT': 3725.772762335},
  'positionSide': {'ETHUSDT': 'LONG'},
  'POSITION_SHARE': {'ETHUSDT': 1.0}},
 'account': {'transfer_balance': -21898.259983239986,
  'historic_PNL': 113715.98